In [3]:
# 关注具有单隐藏层的多层感知机
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(2, 4)
net(X)

tensor([[0.0770],
        [0.0408]], grad_fn=<AddmmBackward0>)

In [4]:
# 参数访问
print(net[2].state_dict())  # 第三层的权重和偏置

OrderedDict([('weight', tensor([[-0.0421,  0.3346, -0.2781, -0.0350,  0.1379,  0.0966,  0.2113, -0.1965]])), ('bias', tensor([-0.0652]))])


In [7]:
# 目标参数
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)  # 访问偏置的数值

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.0652], requires_grad=True)
tensor([-0.0652])


In [10]:
net[2].weight.grad == None  # 未进行反向计算

True

In [11]:
# 一次性访问所有参数
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [12]:
net.state_dict()['2.bias'].data

tensor([-0.0652])

In [15]:
# 从嵌套块收集参数
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        net.add_module(f'block {i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
print(rgnet)
rgnet(X)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


tensor([[-0.4738],
        [-0.4738]], grad_fn=<AddmmBackward0>)

In [16]:
# 内置初始化
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)  # 权重初始化为均值为0, 标准差为0.01的正态分布
        nn.init.zeros_(m.bias)  # 偏差初始化为0

net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([-0.0065,  0.0050, -0.0032, -0.0095]), tensor(0.))

In [17]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)  # 权重初始化为常数1
        nn.init.zeros_(m.bias)  # 偏置初始化为0

net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

In [27]:
# 对某些块应用不同的初始化方法
def xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)  # 权重初始化的方差由输入和输出维度共同决定

def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)

net[0].apply(xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])  # 第一个全连接层共8个输出, 这里取第一个输出对应的权重
print(net[2].weight.data)  # 第二个全连接层只有1个输出, 有8个权重

tensor([-0.5788,  0.1594, -0.1057, -0.2554])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


In [34]:
# 自定义初始化
def my_init(m):
    if type(m) == nn.Linear:
        print(
            "Init",
            *[(name, param.shape) for name, param in m.named_parameters()][0])  # 只取出权重的shape
        nn.init.uniform_(m.weight, -10, 10)  # 均匀初始化
        m.weight.data *= (m.weight.data.abs() >= 5)  # 保留绝对值大于5的权重

net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[-9.4247,  0.0000,  7.4661,  0.0000],
        [ 5.9530,  0.0000, -7.8201, -0.0000]], grad_fn=<SliceBackward0>)

In [37]:
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.0000,  2.0000,  9.4661,  2.0000])

In [38]:
# 参数绑定
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), shared, nn.ReLU(), shared, 
                   nn.ReLU(), nn.Linear(8, 1))
net(X)
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])
